In [ ]:
#extract response amplitudes from each trial
import os
import mne
import numpy as np
from reftep_util_funcs import *
import shutil
responses_dipoles = ['n15','p30','n45','p60']
responses_emg = ['mep']
responses_all = responses_dipoles + responses_emg

for site in ['Tuebingen','Aalto']:
    site_path_sources = fr"D:\REFTEP_ALL\Source_analysis\Source_analysis_{site}"
    site_path_preprocessing = fr"D:\REFTEP_ALL\EEG_preprocessing_data\Data_{site}"
    for subject in os.listdir(site_path_sources):
        subject_source_path = os.path.join(site_path_sources,subject)
        forward = mne.read_forward_solution(os.path.join(subject_source_path,f'{subject}-fwd.fif'),verbose=False) #read forward solution
        #read eeg epochs
        epochs = mne.read_epochs(os.path.join(subject_source_path,f'{subject}_final_eeg_post-epo.fif'),verbose=False, proj=False)
        epochs_data = epochs.get_data(copy=True) #load the epochs data into a separate structure
        n_trials = epochs_data.shape[0] #number of trials
        #read emg epochs"
        emg_epochs = mne.read_epochs_eeglab(os.path.join(site_path_preprocessing,subject,f'{subject}_EMG_processed.set'),verbose=False) #load emg data

        #define paths to save data to
        dipoles_folder_path = os.path.join(subject_source_path,f'{subject}_dipoles')
        mep_ptps_filepath = os.path.join(subject_source_path,f'{subject}_mep_amplitudes')

        for response_type in responses_all:

            if response_type in responses_emg: #get mep response amplitudes and save them
                mep_ptps, maxind = get_mep_ptps(emg_epochs,tmin=0.02, tmax=0.05)
                np.save(mep_ptps_filepath, mep_ptps)
                ch_info_mep = {'channel_used_ind':maxind, 'channel_used_name':emg_epochs.ch_names[maxind]}
                mep_ch_info_ptps_filepath = os.path.join(subject_source_path,f'{subject}_mep_ch_info')
                np.save(mep_ch_info_ptps_filepath, ch_info_mep)
                print(f'{subject}, mep amps mean std, {np.mean(mep_ptps)*1e6}, {np.std(mep_ptps)*1e6} at at channel {emg_epochs.ch_names[maxind]} (maxind {maxind})')

            elif response_type in responses_dipoles: #dipoles separately to emg
                dipole_now = mne.read_dipole(os.path.join(dipoles_folder_path,f'{subject}_dipole_{response_type}')) #filepath for the dipole
                potential_times = np.load(os.path.join(dipoles_folder_path,f'{subject}_dipole_{response_type}_fitting_times.npy'))
                #tmin, tmax = np.min(potential_times), np.max(potential_times) #min and max time of the fitting range
                #number_of_times_to_consider = len(potential_times)
                pos_ind = int(dipole_now.name.split("_")[-1]) #the position of the dipole is saved here

                dipoles_path_now = os.path.join(dipoles_folder_path,f'{subject}_dipole_{response_type}_trials') #define path to save single trial dipoles
                if os.path.exists(dipoles_path_now):
                    shutil.rmtree(dipoles_path_now)
                os.makedirs(dipoles_path_now)
                dipoles_path_now_consider = os.path.join(dipoles_folder_path,f'{subject}_dipole_{response_type}_trials_all_dipoles') #define path to save all single trial dipoles
                if os.path.exists(dipoles_path_now_consider):
                    shutil.rmtree(dipoles_path_now_consider)
                os.makedirs(dipoles_path_now_consider)

                #store the gofs and amps of the best dipoles for each trial
                dipamps = []
                dipgofs = []

                #go through all trials
                for trial_ind in range(n_trials):
                    trial_dipoles_times = [] #for consistency checking
                    current_dipole = None
                    best_r2_for_dipole = -np.inf
                    #create a directory for each trial for saving dipoles for that trial
                    trial_path_dipole = os.path.join(dipoles_path_now_consider, f'trial_{trial_ind}')
                    os.mkdir(trial_path_dipole)
                    for time_ind, time_now in enumerate(potential_times): #fit dipole to position with orientation
                        evoked_trial_now = mne.EvokedArray(data=epochs_data[trial_ind,:,:], info=epochs.info, tmin=epochs.times[0],nave=1, kind='single_epoch',verbose=False)
                        dipole_trial_now, _, _ = dipole_to_pos(forward, evoked_trial_now, time_now, time_now, pos_ind, 1, "r2", ori_fixed=dipole_now.ori[0])
                        trial_dipole_path_now = os.path.join(trial_path_dipole,f'{subject}_dipole_{time_now}')
                        dipole_trial_now.save(trial_dipole_path_now,overwrite=True,verbose=False) #save each dipole at each time point that is considered
                        if dipole_trial_now.times[0] in trial_dipoles_times:
                            print(f"duplicate time observed in structure... in trial {trial_ind} at time {dipole_trial_now.time[0]}")
                            break
                        trial_dipoles_times.append(dipole_trial_now.times[0])
                        if dipole_trial_now.gof[0] > best_r2_for_dipole: #update to the best dipole for this trial
                            best_r2_for_dipole = dipole_trial_now.gof[0]
                            current_dipole = dipole_trial_now
                    dipgofs.append(current_dipole.gof[0])
                    dipamps.append(current_dipole.amplitude[0])
                    trial_dipole_path = os.path.join(dipoles_path_now,f'{subject}_dipole_{response_type}_trial_{trial_ind}')
                    current_dipole.save(trial_dipole_path,overwrite=True,verbose=False) #save the best dipole for each trial separately too, can be later matched to the other folder with comparing times
                print(f'{subject},{response_type}, mean and std of gof and amp {np.mean(dipgofs)}, {np.std(dipgofs)}, {np.mean(dipamps)*1e9}, {np.std(dipamps)*1e9}')
